In [13]:
import pandas as pd
import pandas_ta as ta
import numpy as np



candlestick_frame = 12
pnl_threshold = 3

ohlc_df = pd.read_csv('../../data/ETHUSDT-ohlcv.csv')
ohlc_df['LOGRET_1'] = list(ohlc_df.ta.log_return())
ohlc_df = ohlc_df.dropna()
LOGRET_1 = list(ohlc_df['LOGRET_1'])

dta_lst = []
tot = len(ohlc_df)


def get_sig(df):
    ts = list(df['time'])
    open = list(df['open'])
    high = list(df['high'])
    low = list(df['low'])
    close = list(df['close'])
    volume = list(df['volume'])
    tot = len(ts)
    long_runup_lst = []
    long_drawdown_lst = []
    short_runup_lst = []
    short_drawdown_lst = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            max_high = max(high[idx+1:idx+candlestick_frame])
            min_low = min(low[idx+1:idx+candlestick_frame])
            entry_price = open[idx+1]
            long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
            long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
            short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
            short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
        else:
            long_runup_lst.append(0)
            long_drawdown_lst.append(0)
            short_runup_lst.append(0)
            short_drawdown_lst.append(0)     


    long=[]
    short=[]
    dont_trade=[]
    signal = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            if long_runup_lst[idx] >= pnl_threshold:
                signal.append('long')
            elif short_runup_lst[idx] >= pnl_threshold:
                signal.append('short')
            else:
                signal.append('dont_trade')
        else:
            signal.append('dont_trade')


    # df['signal'] = signal
    return signal[candlestick_frame:]


sig = get_sig(ohlc_df)
for idx in range(candlestick_frame, tot, 1):
    app = {}
    for i in range(candlestick_frame):
        app[i] = LOGRET_1[idx-i]
    dta_lst.append(app)

df = pd.DataFrame(dta_lst)

df.to_csv(f'../../data/df.csv', index=False)
y = pd.DataFrame(sig).to_csv('../../data/y.csv', index=False)

In [8]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np


df = pd.read_csv('../../data/df.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.000108,-0.002288,0.002011,0.001594,0.003397,-0.002540,0.000313,-0.001781,0.000128,-0.001501,0.005098,0.000391
1,-0.002185,-0.000108,-0.002288,0.002011,0.001594,0.003397,-0.002540,0.000313,-0.001781,0.000128,-0.001501,0.005098
2,-0.002808,-0.002185,-0.000108,-0.002288,0.002011,0.001594,0.003397,-0.002540,0.000313,-0.001781,0.000128,-0.001501
3,-0.002857,-0.002808,-0.002185,-0.000108,-0.002288,0.002011,0.001594,0.003397,-0.002540,0.000313,-0.001781,0.000128
4,0.001533,-0.002857,-0.002808,-0.002185,-0.000108,-0.002288,0.002011,0.001594,0.003397,-0.002540,0.000313,-0.001781


In [9]:
len(df)

20500

In [11]:
y = pd.read_csv('../../data/y.csv')
len(y)

20500

In [3]:
df["id"] = df.index
df = df.melt(id_vars="id", var_name="time").sort_values(["id", "time"]).reset_index(drop=True)
df.to_csv('../../data/df_id_time.csv', index=False)
df.head()

,id,time,value
0,0,0,-0.000108
1,0,1,-0.002288
2,0,10,0.005098
3,0,11,0.000391
4,0,2,0.002011


In [4]:
# only use the first 500 ids to speed up the processing
X = extract_features(df[df["id"] < 20000], column_id="id", column_sort="time", impute_function=impute)

Feature Extraction: 100%|██████████| 50/50 [01:16<00:00,  1.53s/it]


In [6]:
len(df)

246000

In [7]:
len(y)

20500

In [30]:
X

,value__variance_larger_than_standard_deviation,value__has_duplicate_max,value__has_duplicate_min,value__has_duplicate,value__sum_values,value__abs_energy,value__mean_abs_change,value__mean_change,value__mean_second_derivative_central,value__median,...,value__fourier_entropy__bins_5,value__fourier_entropy__bins_10,value__fourier_entropy__bins_100,value__permutation_entropy__dimension_3__tau_1,value__permutation_entropy__dimension_4__tau_1,value__permutation_entropy__dimension_5__tau_1,value__permutation_entropy__dimension_6__tau_1,value__permutation_entropy__dimension_7__tau_1,value__query_similarity_count__query_None__threshold_0.0,value__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,0.0,0.0,0.004715,0.000062,0.002958,-0.000127,0.000028,0.000221,...,1.277034,1.475076,1.945910,1.279854,1.676988,1.906155,1.94591,1.791759,0.0,0.002673
1,0.0,0.0,0.0,0.0,0.002139,0.000066,0.003343,0.000210,-0.000008,0.000010,...,1.277034,1.475076,1.945910,1.366159,1.735126,1.906155,1.94591,1.791759,0.0,0.002757
2,0.0,0.0,0.0,0.0,-0.005767,0.000048,0.002322,0.000093,-0.000136,-0.000805,...,1.277034,1.747868,1.945910,1.557113,2.197225,2.079442,1.94591,1.791759,0.0,0.002430
3,0.0,0.0,0.0,0.0,-0.007123,0.000054,0.002261,0.000288,0.000140,-0.000944,...,1.277034,1.747868,1.945910,1.557113,2.197225,2.079442,1.94591,1.791759,0.0,0.002584
4,0.0,0.0,0.0,0.0,-0.005718,0.000056,0.002547,-0.000370,-0.000077,-0.000944,...,0.955700,1.153742,1.945910,1.748067,2.197225,2.079442,1.94591,1.791759,0.0,0.002584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,-0.017207,0.000500,0.006910,0.001138,-0.000281,-0.000306,...,1.475076,1.747868,1.945910,1.748067,2.197225,2.079442,1.94591,1.791759,0.0,0.006597
9996,0.0,0.0,0.0,0.0,-0.007805,0.000563,0.008351,-0.001007,0.001603,0.000760,...,1.475076,1.747868,1.945910,1.470808,2.043192,2.079442,1.94591,1.791759,0.0,0.007305
9997,0.0,0.0,0.0,0.0,-0.007462,0.000563,0.008264,-0.001797,-0.001464,0.000931,...,0.796312,1.277034,1.945910,1.504788,1.889159,2.079442,1.94591,1.791759,0.0,0.007305
9998,0.0,0.0,0.0,0.0,-0.004869,0.000591,0.009030,-0.000255,0.000427,0.000931,...,0.955700,1.153742,1.747868,1.470808,2.043192,2.079442,1.94591,1.791759,0.0,0.007675
